# Prototype of Droneport Orchestrator

Droneport is a system for autonomous drone battery management. It consists of both hardware and software components. One of the software components is Droneport Traffic Control, which takes care of controlling the whole system; i.e., it sends commands to both drones and ports. Part of this component is the so-called Droneport Ochestrator, whose purpose is to schedule the drone's battery replacement with respect to the specified missions and battery status.

In the current version, Droneport Orchestrator is based on the planner from the article
>Song, B. D., Kim, J., & Morrison, J. R. (2016). Rolling Horizon Path Planning of an Autonomous System of UAVs for Persistent Cooperative Service: MILP Formulation and Efficient Heuristics. Journal of Intelligent & Robotic Systems, 84(1–4), 241–258. https://doi.org/10.1007/s10846-015-0280-5,

nevertheless further extensions are possible in the future.

## Notation

The notation is the same as in the article

|symbol|meaning| 
|------|-------|
|i,j|Indices for jobs|
|$s$|Index for stations| 
|$k$|Index for UAVs| 
|$r$|Index of a UAV’s $r^th$ flight| 
|$N_J$|Number of split jobs| 
|$N_{UAV}$|Number of UAVs in the system| 
|$N_{STA}$|Number of recharge stations| 
|$N_R$|Maximum number of flights per UAV during the time horizon| 
|$M$|Large positive number| 
|$D_{ij}$|Distance from the finish point of split job $i$ to the start point of split job $j$| 
|$E_i$|Start time of split job $i$|
|$L_i$|End time of split job $i$|
|$P_i$|Processing time of split job $i$ $(Li−Ei)$|
|$H$|Required time for fully recharge (refuel) the empty fuel tank (battery)|
|$U$|Setup time for recharge/refuel process|
|$q_k$|Maximum traveling time of UAV $k$|
|$q_{k,ini}$|Initial level of battery(fuel) of UAV $k$|
|$TS_k$|Travel speed of UAVt $k$|
|$w_1$|Weight factor between objective criteria, $0≤
w_1 ≤1$|
|$w_2$|Positive integer Scaling factor on the number of served jobs in objective function|
|$Ω_J$|={0, …, $N_J-1$}, Set of split jobs|
|$Ω_{SS}$|={$N_{J}, N_{J}+2, …, N_{J}+2⋅N_{STA}−2$}, set of UAV flight start stations|
|$Ω_{SE}$|={$N_{J}+1, N_{J}+3, …, N_{J}+2⋅N_{STA}-1$}, set of UAV flight end stations|
|$Ω_A$|=($Ω_J U Ω_{SS} UΩ_{SE}$)={$0,…, N_{J}+2⋅N_{STA}-1$}, set of all jobs and recharge stations|
|$Ω_{INI}$|={$0_{INI}, …, (K-1)_{INI}$}, set of initial UAV location|
|$X_{ijkr}$|Binary decision variable, 1 if UAV $k$ processes split job $j$ or recharges at station $j$ after processing split job $i$ or recharging at station $i$ during the $r^th$ flight; 0, otherwise.|
|$C_{ikr}$|Real number decision variable, split job $i$’s start time by UAV $k$ during its $r^th$ flightor UAV $k$’s recharge start time at station $i$; otherwise its value is 0|
|$q_{kr}$|Real number decision variable, total battery (fuel) consumption for UAV $k$ during its $r^th$ flight|

# MILP implementation in Pyomo

Some parameters are initialized with NumPy matrices. The matplotlib and tikzplotlib libraries are used for visualization and subsequent export. The optimization problem is described and solved using the Pyomo package.

In [56]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import tikzplotlib as tplt
from pyomo.environ import *
from gurobipy import GRB 
from pyomo.contrib import appsi
import pandas as pd

Variables and parameters are named according to the notation above.

In [66]:
def distance(a, b):
    return np.linalg.norm(b-a,2)
    
# init model
m = ConcreteModel()

# count
m.Njob = 15 # no. of split jobs
m.Ndrone = 6 # no. of drones
m.Nport = 3 # no. of Droneports
m.Nr = 5 # max no. of flights per drone on the time horizon
m.M = 100 # "large positive number"

# sets
m.set_job = RangeSet(0,m.Njob-1) # Ωj
m.set_start_port = RangeSet(m.set_job.at(-1)+1,m.set_job.at(-1)+m.Nport) # Ωss
m.set_end_port = RangeSet(m.set_start_port.at(-1)+1,m.set_start_port.at(-1)+m.Nport) # Ωse
# m.set_ini = RangeSet(m.set_end_port.at(-1),m.set_end_port.at(-1)+m.Ndrone) # Ωini
m.set_drone = RangeSet(0,m.Ndrone-1) # K
m.set_flight = RangeSet(0,m.Nr-1) # R
m.set_flight_w1st = RangeSet(1,m.Nr-1) # R without 1st element
m.set_flight_w1st_wend = RangeSet(1,m.Nr-2) # R without 1st element shorter

# input data
jobs = {'start_x': [596, 532, 438, 432, 372, 315, 262, 218, 171, 142, 102, 54, 458, 479, 514],
        'start_y': [167, 161, 129, 94, 91, 87, 74, 99, 134, 186, 225, 251, 64, 105, 63],
        'end_x': [532, 483, 432, 372, 315, 262, 218, 171, 142, 102, 54, 6, 479, 514, 536],
        'end_y': [161, 129, 94, 91, 87, 74, 99, 134, 186, 225, 251, 266, 105, 63, 15],
        'start_t': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 8, 9, 10]}
port = {'x': [394, 170, 72],
            'y': [126, 79, 229]}
drone = {'x': [560, 280, 170, 300, 394, 90],
       'y': [130, 180, 79, 100, 126, 200],
       'q_k': [12, 12, 12, 12, 12, 12],
       'q_ini': [3, 8, 8, 6, 6, 8],
       'TS_k': [240, 240, 240, 240, 240, 240]}
df_port = pd.DataFrame(port)
df_drone = pd.DataFrame(drone)
df_jobs = pd.DataFrame(jobs)
df_jobs['end_t'] = df_jobs['start_t']+1

start_position = [-1]*m.Ndrone
# find drones starting on the Droneport
n = 0
for k, drone in df_drone.iterrows():
    for i, port in df_port.iterrows():
        if drone['x'] == port['x']:
            if drone['y'] == port['y']:
                start_position[k] = i+m.set_start_port.at(1)
    if start_position[k] == -1:
        start_position[k] = n+m.set_end_port.at(-1)+1
        n = n+1

m.set_ini = Set(initialize=start_position) # Ωini
m.set_all = m.set_job | m.set_start_port | m.set_end_port | m.set_ini # Ωa

H = 3
U = 0.5
w1 = 0.3
w2 = 50

# conversion to multi-level dictionary
data = {}
data['D_ij'] = {}
for i in m.set_all:
    if i in m.set_job:
        a = df_jobs.loc[i,['start_x','start_y']].values
    elif i in m.set_start_port:
        a = df_port.loc[i-m.set_start_port.at(1), ['x','y']].values
    elif i in m.set_ini:
        a = df_drone.loc[i-m.set_ini.at(1), ['x','y']].values
    else:
        for j in m.set_all:
            data['D_ij'][i,j] = 1e6
        continue
    for j in m.set_all:
        if j in m.set_job:
            b = df_jobs.loc[j,['end_x','end_y']].values
        elif j in m.set_end_port:
            b = df_port.loc[j-m.set_end_port.at(1), ['x','y']].values
        else:
            data['D_ij'][i,j] = 1e6
            continue
        data['D_ij'][i,j] = distance(a,b)

data['E_i'] = {}
for i in m.set_job:
    data['E_i'][i] = df_jobs['start_t'][i]
data['L_i'] = {}
for i in m.set_job:
    data['L_i'][i] = df_jobs['end_t'][i]
data['P_i'] = {}
for i in m.set_job:
    data['P_i'][i] = df_jobs['end_t'][i] - df_jobs['start_t'][i]
data['H'] = H
data['U'] = U
data['q_k'] = {}
for i in m.set_drone:
    data['q_k'][i] = df_drone['q_k'][i]
data['q_ini'] = {}
for i in m.set_drone:
    data['q_ini'][i] = df_drone['q_ini'][i]
data['TS_k'] = {}
for i in m.set_drone:
    data['TS_k'][i] = df_drone['TS_k'][i]
data['w1'] = w1
data['w2'] = w2

# parameters
m.D_ij = Param(m.set_all, m.set_all, domain=NonNegativeReals, initialize=data['D_ij']) # distance between jobs
m.E_i = Param(m.set_job, domain=NonNegativeReals, initialize=df_jobs['start_t']) # start times
m.L_i = Param(m.set_job, domain=NonNegativeReals, initialize=data['L_i']) # end times
m.P_i = Param(m.set_job, domain=NonNegativeReals, initialize=data['P_i']) # diff times
m.H = Param(domain=PositiveReals, initialize=data['H']) # recharge time
m.U = Param(domain=PositiveReals, initialize=data['U']) # setup time
m.q_k = Param(m.set_drone, domain=PositiveReals, initialize=data['q_k']) # max travel time
m.q_ini = Param(m.set_drone, domain=PositiveReals, initialize=data['q_ini']) # init batt lvl
m.TS_k = Param(m.set_drone, domain=PositiveReals, initialize=data['TS_k']) # travel speeds
m.w1 = Param(domain=Reals, initialize=data['w1']) # w1 in <0,1>
m.w2 = Param(domain=PositiveIntegers, initialize=data['w2']) # w2 positive integer

# variables
m.X_ijkr = Var(m.set_all, m.set_all, m.set_drone, m.set_flight, domain=Binary) # split-job OR recharge
m.C_ikr = Var(m.set_all, m.set_drone, m.set_flight, domain=NonNegativeIntegers) # split-job start time 
m.q_kr = Var(m.set_drone, m.set_flight, domain=Reals) # battery consumption

# auxiliary variables
m.RTf = Var(m.set_drone,domain=Reals) # remaining fuel before first Droneport visit
m.RTr = Var(m.set_drone,domain=Reals) # remaining fuel after first Droneport visit

IndexError: list assignment index out of range

The constraints and objective function of the MILP are indicated by the equation numbers from the corresponding article. Several errors and ambiguities in the code have been corrected. 

In [58]:
# auxiliary variable rules
# Eq. 1 RT_f (H/q_k)*(q_{kr-1} + q_k - q_k,ini) + U
m.eq1_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight_w1st:
        m.eq1_const.add(m.RTf[k] == (m.H/m.q_k[k])*(m.q_kr[k,r-1] + m.q_k[k] - m.q_ini[k]) + m.U)
# Eq. 2 RT_r = (H/q_k)*q_{kr-1} + U
m.eq2_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight_w1st:
        m.eq2_const.add(m.RTr[k] == (m.H/m.q_k[k])*m.q_kr[k,r-1] + m.U)

# constraints
# Eq. 12 Sum_{i in Omega_ss U Omega_ini} Sum_{j in Omega_j U Omega_se} X_ijkr = 1 (k in K, r in R)
m.eq12_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight:
        m.eq12_const.add(sum(sum(m.X_ijkr[i,j,k,r] for j in m.set_job | m.set_end_port) for i in m.set_start_port | m.set_ini) == 1)
# Eq. 13 Sum_{i in Omega_j U Omega_ss U Omega_ini} Sum_{j in Omega_j U Omega_se} X_iskr = 1 (k in K, r in R)
m.eq13_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight:
        m.eq13_const.add(sum(sum(m.X_ijkr[i,s,k,r] for s in m.set_end_port) for i in m.set_job | m.set_start_port | m.set_ini) == 1)
# Eq. 14 Sum_{i in Omega_j U Omega_ss} X_ijkr = Sum_{i in Omega_j U Omega_se} X_s-1, ikr+1 (k in K, r = 1,...,N_r-1, s in Omega_se)
m.eq14_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight_w1st_wend: # Error in paper (1,...,Nr-1)
        for s in m.set_end_port:
            m.eq14_const.add(sum(m.X_ijkr[i,s,k,r] for i in m.set_job | m.set_start_port) ==
                             sum(m.X_ijkr[s-m.Nport,i,k,r+1] for i in m.set_job | m.set_end_port))
# Eq. 15 Sum_{i in Omega_j U Omega_ss} X_ijkr = 0 (k in K, r in R, s in Omega_ss U Omega_ini)
m.eq15_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight_w1st:
        for s in m.set_start_port | m.set_ini:
            m.eq15_const.add(sum(m.X_ijkr[i,s,k,r] for i in m.set_job | m.set_start_port) == 0)
# Eq. 16 Sum_{k in K} Sum_{r in R} Sum_{i in Omega_a} X_ijkr <= 1 (j in Omega_j)
m.eq16_const = ConstraintList()
for j in m.set_job:
    m.eq16_const.add(sum(sum(sum(m.X_ijkr[i,j,k,r] for i in m.set_all) for r in m.set_flight) for k in m.set_drone) <= 1)
# Eq. 17 Sum_{j in Omega_a} X_ijkr - Sum_{j in Omega_a} X_jikr = 0 (i in Omega_j, k in K, r in R)
m.eq17_const = ConstraintList()
for i in m.set_job:
    for k in m.set_drone:
        for r in m.set_flight:
            m.eq17_const.add(sum(m.X_ijkr[i,j,k,r] for j in m.set_all) - sum(m.X_ijkr[i,j,k,r] for j in m.set_all) == 0)
# Eq. 18 C_skr = C_{s−1,kr+1} (k in K, r = 1,...N_r − 1, s in Omega_se)
m.eq18_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight_w1st_wend:
        for s in m.set_end_port:
            m.eq18_const.add(m.C_ikr[s,k,r] == m.C_ikr[s-m.Nport,k,r+1])
# Eq. 19 C_ikr + P_i + D_ij/TS_k − C_jkr <= M (1−X_ijkr) (i in Omega_j, j in Omega_j U Omega_se, k in K, r in R)
m.eq19_const = ConstraintList()
for i in m.set_job:
    for j in m.set_job | m.set_end_port:
        for k in m.set_drone:
            for r in m.set_flight:
                m.eq19_const.add(m.C_ikr[i,k,r] + m.P_i[i] + m.D_ij[i,j]/m.TS_k[k] - m.C_ikr[j,k,r] <= m.M*(1-m.X_ijkr[i,j,k,r]))
# Eq. 20 M Sum_{jijkr} >= C_ikr (i in Omega_j U Omega_ss, k in K, r in R)
m.eq20_const = ConstraintList()
for i in m.set_job | m.set_start_port:
    for k in m.set_drone:
        for r in m.set_flight:
            m.eq20_const.add(m.M*sum(m.X_ijkr[i,j,k,r] for j in m.set_job | m.set_end_port) >= m.C_ikr[i,k,r])
# Eq. 21 C_ikr + RT_f + D_ij/TS_k - C_jkr <= M(1-X_ijkr) (i in Omega_ss, j in Omega_j U Omega_se, k in K, r = 2)
m.eq21_const = ConstraintList()
r = 1
for i in m.set_start_port:
    for j in m.set_job | m.set_end_port:
        for k in m.set_drone:
            m.eq21_const.add(m.C_ikr[i,k,r] + m.RTf[k] + m.D_ij[i,j]/m.TS_k[k] - m.C_ikr[j,k,r] <= m.M*(1-m.X_ijkr[i,j,k,r]))
# Eq. 22 C_ikr + RT_r + D_ij/TS_k - C_jkr <= M(1-X_ijkr) (i in Omega_ss, j in Omega_j U Omega_se, k in K, r > 2)
m.eq22_const = ConstraintList()
for i in m.set_start_port:
    for j in m.set_job | m.set_end_port:
        for k in m.set_drone:
            for r in RangeSet(2,m.Nr-1):
                m.eq22_const.add(m.C_ikr[i,k,r] + m.RTr[k] + m.D_ij[i,j]/m.TS_k[k] - m.C_ikr[j,k,r] <= m.M*(1-m.X_ijkr[i,j,k,r]))
# Eq. 23 Sum_{k in K} Sum_{r in R} C_ikr = E_i (i in Omega_j)
m.eq23_const = ConstraintList()
for i in m.set_job:
    m.eq23_const.add(sum(sum(m.C_ikr[i,k,r] for r in m.set_flight) for k in m.set_drone) == m.E_i[i])
# Eq. 24 Sum_{i in Omega_a} Sum_{j in Omega_a} D_ij/TS_k X_ijkr + Sum_{i in Omega_j} Sum_{j in Omega_a} P_i X_ijkr <= q_kr (k in K, r in R)
m.eq24_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight:
        m.eq24_const.add(sum(sum((m.D_ij[i,j]/m.TS_k[k])*m.X_ijkr[i,j,k,r] for j in m.set_all) for i in m.set_all)
                        + sum(sum(m.P_i[i]*m.X_ijkr[i,j,k,r] for j in m.set_all) for i in m.set_job) <= m.q_kr[k,r])
# Eq. 25 q_kr <= q_k,ini (k in K, r = 1)
m.eq25_const = ConstraintList()
r = 0
for k in m.set_drone:
    m.eq25_const.add(m.q_kr[k,r] <= m.q_ini[k])
# Eq. 26 q_kr <= q_k (k in K, r != 1)
m.eq26_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight_w1st:
        m.eq26_const.add(m.q_kr[k,r] <= m.q_k[k])
# Eq. 27 C_ikr >= 0 (k in K, r in R, i in Omega_a)
# met by definition of variable

# Eq. 28 q_kr >= 0 (k in K, r in R)
m.eq28_const = ConstraintList()
for k in m.set_drone:
    for r in m.set_flight:
        m.eq28_const.add(m.q_kr[k,r] >= 0)
# Eq. 29 X_ijkr in {0,1} (i in Omega_a, j in Omega_a, k in K, r in R)
# met by definition of variable

# r = 0
# m.eqX_const = ConstraintList()
# for k in m.set_drone:
#     for s in m.set_ini:
#         m.eqX_const.add(sum(m.X_ijkr[s,j,k,r] for j in m.set_job | m.set_end_port) == 1)

# objective
# Eq. 11 w_1 Sum_{i in Omega_a} Sum_{j in Omega_a} Sum_{k in K} Sum_{r in R} D_ij X_ijkr + (1-w1) w2 (abs(J) - Sum_{i in Omega_j} Sum_{j in Omega_j U Omega_se} Sum_{k in K} Sum_{r in R} X_ijkr)
m.total_cost = 0
m.total_cost = m.total_cost + m.w1*sum(sum(sum(sum(m.D_ij[i,j]*m.X_ijkr[i,j,k,r] for r in m.set_flight) for k in m.set_drone) for j in m.set_all) for i in m.set_all) + (1-m.w1)*m.w2*(abs(m.Njob)- sum(sum(sum(sum(m.X_ijkr[i,j,k,r] for r in m.set_flight) for k in m.set_drone) for j in m.set_job | m.set_end_port)for i in m.set_job))

m.obj = Objective(expr = m.total_cost, sense=minimize)

In [59]:
# # Should print all available solvers
# #print(sorted(IOptSolver._factory_cls.keys()))
# opt = SolverFactory("gurobi", solver_io="python") # pip install gurobipy needed
# # Create a model instance and optimize
# instance = m
# results = opt.solve(instance)
# instance.display()


opt = appsi.solvers.Gurobi()
opt.config.stream_solver = True
opt.set_instance(m)
res = opt.solve(m)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 10890 rows, 19542 columns and 80196 nonzeros
Model fingerprint: 0x06c29096
Variable types: 42 continuous, 19500 integer (18750 binary)
Coefficient statistics:
  Matrix range     [8e-02, 4e+03]
  Objective range  [2e-01, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+02]
Presolve removed 2151 rows and 10293 columns
Presolve time: 0.16s
Presolved: 8739 rows, 9249 columns, 55033 nonzeros
Variable types: 6 continuous, 9243 integer (8739 binary)

Root relaxation: objective 5.030971e+02, 2031 iterations, 0.10 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  503.09709    0    6          -  503.09709      -     -    0s
H    0     0                     638.0976739  503.09709  

In [60]:
drone_plan = {}
for k in m.set_drone:
    drone_plan[k] = {}
    for r in m.set_flight:
        for j in m.set_all:
            for i in m.set_all:
                if m.X_ijkr[i,j,k,r].value == 1:
                    if i in m.set_job:
                        drone_plan[k][r] = i
                    elif i in m.set_start_port:
                        drone_plan[k][r] = 'S{0}'.format((i-m.set_start_port.at(1)))
                    elif i in m.set_ini:
                        drone_plan[k][r] = 'I{0}'.format(i-m.set_ini.at(1))
for k in m.set_drone:
    print(drone_plan[k])

{0: 3, 1: 'I2', 2: 'I2', 3: 'I2', 4: 8}
{0: 10, 1: 'I2', 2: 4, 3: 'S0', 4: 'S0'}
{0: 'S2', 1: 'I2', 2: 'I2', 3: 11, 4: 'S2'}
{0: 5, 1: 'S1', 2: 'S1', 3: 'S1', 4: 'S1'}
{0: 2, 1: 'I2', 2: 'I2', 3: 'I2', 4: 13}
{0: 7, 1: 'I2', 2: 'I2', 3: 'I2', 4: 14}


In [61]:
drone_plan_end = {}
for k in m.set_drone:
    drone_plan_end[k] = {}
    for r in m.set_flight:
        for j in m.set_all:
            for i in m.set_all:
                if m.X_ijkr[i,j,k,r].value == 1:
                    if j in m.set_job:
                        drone_plan_end[k][r] = j
                    elif j in m.set_end_port:
                        drone_plan_end[k][r] = 'S{0}'.format((j-m.set_end_port.at(1)))
                    elif j in m.set_ini:
                        drone_plan_end[k][r] = 'I{0}'.format(j-m.set_ini.at(1))
                        
for k in m.set_drone:
    print(drone_plan_end[k])

{0: 'S0', 1: 'S1', 2: 'S1', 3: 'S1', 4: 'S1'}
{0: 'S2', 1: 'S1', 2: 'S0', 3: 'S0', 4: 'S0'}
{0: 'S2', 1: 'S1', 2: 'S1', 3: 'S2', 4: 'S2'}
{0: 'S0', 1: 'S1', 2: 'S1', 3: 'S1', 4: 'S1'}
{0: 'S0', 1: 'S1', 2: 'S1', 3: 'S1', 4: 'S0'}
{0: 'S1', 1: 'S1', 2: 'S1', 3: 'S1', 4: 'S0'}


In [62]:
drone_battery = {}
for k in m.set_drone:
    drone_battery[k] = {}
    for r in m.set_flight:
        drone_battery[k][r] = m.q_kr[k,r].value
        
for k in m.set_drone:
    print(drone_battery[k])

{0: 1.3792459275445212, 1: 1.3792459275445212, 2: 1.3792459275445212, 3: 1.3792459275445212, 4: 12.0}
{0: 1.4014547640628638, 1: 1.4014547640628638, 2: 1.4014547640628638, 3: 1.4014547640628638, 4: 12.0}
{0: 2.858580989312122, 1: 2.858580989312122, 2: 2.858580989312122, 3: 2.858580989312122, 4: 12.0}
{0: 1.4931987719454547, 1: 1.4931987719454547, 2: 1.4931987719454547, 3: 1.4931987719454547, 4: 12.0}
{0: 1.3021978956229283, 1: 1.3021978956229283, 2: 1.3021978956229283, 3: 1.3021978956229283, 4: 12.0}
{0: 1.3907216547850378, 1: 1.3907216547850378, 2: 1.3907216547850378, 3: 1.3907216547850378, 4: 12.0}


In [63]:
job_start = {}
for k in m.set_drone:
    job_start[k] = {}
    for r in m.set_flight:
        job_start[k][r] = {}
        for i in m.set_job:
            if m.C_ikr[i,k,r].value > 0:
                job_start[k][r][i] = "drone {0}, flight {1}, job {2}: {3}".format(k, r, i, m.C_ikr[i,k,r].value)
                
for k in m.set_drone:
    print(job_start[k])

{0: {3: 'drone 0, flight 0, job 3: 8.0'}, 1: {}, 2: {}, 3: {}, 4: {6: 'drone 0, flight 4, job 6: 11.0', 8: 'drone 0, flight 4, job 8: 13.0'}}
{0: {10: 'drone 1, flight 0, job 10: 15.0'}, 1: {}, 2: {4: 'drone 1, flight 2, job 4: 9.0'}, 3: {}, 4: {}}
{0: {}, 1: {}, 2: {}, 3: {9: 'drone 2, flight 3, job 9: 14.0', 11: 'drone 2, flight 3, job 11: 16.0'}, 4: {}}
{0: {5: 'drone 3, flight 0, job 5: 10.0'}, 1: {}, 2: {}, 3: {}, 4: {}}
{0: {2: 'drone 4, flight 0, job 2: 7.0'}, 1: {}, 2: {}, 3: {}, 4: {0: 'drone 4, flight 4, job 0: 5.0', 13: 'drone 4, flight 4, job 13: 9.0'}}
{0: {7: 'drone 5, flight 0, job 7: 12.0'}, 1: {}, 2: {}, 3: {}, 4: {1: 'drone 5, flight 4, job 1: 6.0', 12: 'drone 5, flight 4, job 12: 8.0', 14: 'drone 5, flight 4, job 14: 10.0'}}


In [64]:
m.set_ini.pprint()
m.set_start_port.pprint()
(m.set_ini -m.set_start_port).pprint()

set_ini : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    6 : {21, 22, 16, 23, 15, 24}
set_start_port : Dimen=1, Size=3, Bounds=(15, 17)
    Key  : Finite : Members
    None :   True : [15:17]
SetDifference_OrderedSet : Size=1, Index=None, Ordered=True
    Key  : Dimen : Domain                   : Size : Members
    None :     1 : set_ini - set_start_port :    4 : {21, 22, 23, 24}
